## Load data & package

In [ ]:
# atualizar variaveis (adicionar novas variaveis de interesse e dar push)!!

df_2019 <- read.csv("./filtered_data2019.csv")
df_2018 <- read.csv("./filtered_data2018.csv")
df_2017 <- read.csv("./filtered_data2017.csv")
df_2016 <- read.csv("./filtered_data2016.csv")
df_2015 <- read.csv("./filtered_data2015.csv")

## Explorative Analysis

In [ ]:
# estabelecer conjunto de variaveis:

mother_profile <- c("")

gest_outcome <- c("")

### Statistic description

avaliar max/min/media/sd das variaveis para cada ano

In [ ]:
summary(df_2015)
# hist(df_2015$IDADEMAE, breaks = c(10:99))

In [ ]:
summary(df_2016)

In [ ]:
summary(df_2017)

In [ ]:
summary(df_2018)

In [ ]:
summary(df_2019)

In [ ]:
# gravidez na infancia/adolescencia
length(which(df_2015$IDADEMAE == 11|df_2015$IDADEMAE == 12|df_2015$IDADEMAE == 13|df_2015$IDADEMAE == 14|df_2015$IDADEMAE == 15))
length(which(df_2016$IDADEMAE == 11|df_2016$IDADEMAE == 12|df_2016$IDADEMAE == 13|df_2016$IDADEMAE == 14|df_2016$IDADEMAE == 15))
length(which(df_2017$IDADEMAE == 11|df_2017$IDADEMAE == 12|df_2017$IDADEMAE == 13|df_2017$IDADEMAE == 14|df_2017$IDADEMAE == 15))
length(which(df_2018$IDADEMAE == 11|df_2018$IDADEMAE == 12|df_2018$IDADEMAE == 13|df_2018$IDADEMAE == 14|df_2018$IDADEMAE == 15))
length(which(df_2019$IDADEMAE == 11|df_2019$IDADEMAE == 12|df_2019$IDADEMAE == 13|df_2019$IDADEMAE == 14|df_2019$IDADEMAE == 15))

In [ ]:
?hist

### Clustering by mother conditions/profile

In [ ]:
# qual estratégia de clustering usar?

### Clustering by gestational outcome

## Predective Analysis

### SHAP

In [ ]:
# avaliar quais parametros melhor predizem o desfecho